<a href="https://colab.research.google.com/github/Hyun-Jun-Lee/News_Analysis/blob/main/text_analysist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from collections import Counter
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt
import datetime

In [3]:
stopwords = pd.read_csv('/content/drive/MyDrive/stopword.csv')
stopwords.sample(10)

,stopdword
10,의해
477,너희들
640,아니
173,과연
421,그런즉
176,생각한대로
205,흥
317,까지
605,중의하나
5,어


In [4]:
data = pd.read_csv('/content/drive/MyDrive/1-8월 뉴스.csv')
data

,title,text,url,media,date
0,무너지는 일과 삶의 경계···노동이 녹아내린다,이 땅에서 ‘비정규직’이라는 표현이 광범위하게 쓰인 지 20여년. 정부가 신규 일자...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01
1,일자리 아닌 일거리···'일정치 않음'을 살다 [녹아내리는 노동],ㆍ①일자리보다 일거리\n\n노동의 미래를 모두 알 수는 없다. 다만 현실에서 일어나...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01
2,이곳은…기계의 정확도 높이는 ‘인간부품’ 육성공장 아닐까 [녹아내리는 노동],ㆍ①일자리보다 일거리\n\nㆍAI 학습데이터 가공 기업 ‘크라우드웍스’ 교육장 참관...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01
3,"지혜씨, 지훈씨를 부탁해! [인포그래픽]",새로 태어나는 아이들은 줄어들고 어르신들은 계속 나이를 먹어 갑니다. 2020년 새...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01
4,"[20대 리포트] “20대, 함부로 판단말라… 내 길 내가 간다”",새천년의 시작과 함께 세상에 나온 2000년대생들이 어느덧 사회 변화를 이끌 20대...,https://news.naver.com/main/read.nhn?mode=LPOD...,국민일보,2020-01-01
...,...,...,...,...,...
29036,"""2.5단계로 무급휴가 요구""… 코로나 재유행에 살아난 '갑질'",직장인 A씨는 수도권 사회적 거리두기 2.5단계를 시행하는 30일부터 다음달 6일...,https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-31
29037,모호한 조치ㆍ 깜깜이 환자 비율 40%육박... 거리두기 2.5단계 효과 있을까,"30일 0시부터 서울 전역에 강화된 사회적 거리두기(2.5단계)가 시행됐지만, 전문...",https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-31
29038,내일부터 서울 시내버스 밤 9시 이후 20% 감축… 주말 수준으로,수도권의 ‘사회적 거리두기’ 2.5단계 시행 기간에 맞춰 서울 시내버스의 야간 운행...,https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-31
29039,집단휴업 두고 전공의 vs 정부 '강대강 대치'... 의료공백 장기화 불가피,21일부터 10일째 무기한 집단휴진을 이어가고 있는 전공의(인턴ㆍ레지던트)들과 정부...,https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-31


In [37]:
data.isna().sum()

title    0
text     0
url      0
media    0
date     0
dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29041 entries, 0 to 29040
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   29041 non-null  object
 1   text    29041 non-null  object
 2   url     29041 non-null  object
 3   media   29041 non-null  object
 4   date    29041 non-null  object
dtypes: object(5)
memory usage: 1.1+ MB


In [36]:
data['media'].value_counts()

경향신문    3715
세계일보    3636
조선일보    3435
동아일보    3316
한국일보    3073
문화일보    2891
서울신문    2817
국민일보    2320
한겨레     2276
중앙일보    1562
Name: media, dtype: int64

In [26]:
# 'date' 컬럼 datetime 형으로 변환 후 월일 형태로
data['date'] = pd.to_datetime(data['date'])
data['date'] = data['date'].dt.strftime('%m/%d')
data.sample(5)

,title,text,url,media,date
16540,올 경제성장률 0.2% ‘사실상 멈춤’… 증세 논의 필요성 커져,"KDI, 2019년 말 2.1%서 대폭 하향 / 상반기 수출·내수 위축 극심해 / ...",https://news.naver.com/main/read.nhn?mode=LPOD...,세계일보,05/21
19888,‘강경 투쟁’ 배수진 쳤지만…묘수 없는 통합당,"국회 일정 보이콧 파장 / 강경파 “민주, 선심 쓰듯이 협상 / 법사위 축소·개편안...",https://news.naver.com/main/read.nhn?mode=LPOD...,세계일보,06/17
4988,"여론 떠밀려 ‘칼럼 고발’ 취하한 與, 사과는 안해",당 비판 칼럼 쓴 교수 고발 논란에… 민주 “과도함 인정” 유감만 표명\n언론중재위...,https://news.naver.com/main/read.nhn?mode=LPOD...,동아일보,02/15
8168,‘확진’ 콜센터 직원 지난달 말에 첫 증상… 열흘 넘게 감염에 노출,최초 확진자 이전 ‘증상 환자’ 확인… 발병일 분석해 최초 감염자 추적\n\n대중교...,https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,03/12
20885,“빨갱이 가족이란 낙인 두려워…금정굴 쪽은 쳐다도 안 봤어요”,[서울신문]\n“어려서는 내가 금정굴을 하루이틀 걸러 다니면서 무섭지도 않은지 거길...,https://news.naver.com/main/read.nhn?mode=LPOD...,서울신문,06/25


In [24]:
data['date'].dt.strftime('%m/%d')

0        01/01
1        01/01
2        01/01
3        01/01
4        01/01
         ...  
29036    08/31
29037    08/31
29038    08/31
29039    08/31
29040    08/31
Name: date, Length: 29041, dtype: object

In [18]:
data

,title,text,url,media,date
0,무너지는 일과 삶의 경계···노동이 녹아내린다,이 땅에서 ‘비정규직’이라는 표현이 광범위하게 쓰인 지 20여년. 정부가 신규 일자...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01
1,일자리 아닌 일거리···'일정치 않음'을 살다 [녹아내리는 노동],ㆍ①일자리보다 일거리\n\n노동의 미래를 모두 알 수는 없다. 다만 현실에서 일어나...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01
2,이곳은…기계의 정확도 높이는 ‘인간부품’ 육성공장 아닐까 [녹아내리는 노동],ㆍ①일자리보다 일거리\n\nㆍAI 학습데이터 가공 기업 ‘크라우드웍스’ 교육장 참관...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01
3,"지혜씨, 지훈씨를 부탁해! [인포그래픽]",새로 태어나는 아이들은 줄어들고 어르신들은 계속 나이를 먹어 갑니다. 2020년 새...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01
4,"[20대 리포트] “20대, 함부로 판단말라… 내 길 내가 간다”",새천년의 시작과 함께 세상에 나온 2000년대생들이 어느덧 사회 변화를 이끌 20대...,https://news.naver.com/main/read.nhn?mode=LPOD...,국민일보,2020-01-01
...,...,...,...,...,...
29036,"""2.5단계로 무급휴가 요구""… 코로나 재유행에 살아난 '갑질'",직장인 A씨는 수도권 사회적 거리두기 2.5단계를 시행하는 30일부터 다음달 6일...,https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-31
29037,모호한 조치ㆍ 깜깜이 환자 비율 40%육박... 거리두기 2.5단계 효과 있을까,"30일 0시부터 서울 전역에 강화된 사회적 거리두기(2.5단계)가 시행됐지만, 전문...",https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-31
29038,내일부터 서울 시내버스 밤 9시 이후 20% 감축… 주말 수준으로,수도권의 ‘사회적 거리두기’ 2.5단계 시행 기간에 맞춰 서울 시내버스의 야간 운행...,https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-31
29039,집단휴업 두고 전공의 vs 정부 '강대강 대치'... 의료공백 장기화 불가피,21일부터 10일째 무기한 집단휴진을 이어가고 있는 전공의(인턴ㆍ레지던트)들과 정부...,https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-08-31


In [5]:
# 필요한 column만
df = data[['title','text','date']]
df

,title,text,date
0,무너지는 일과 삶의 경계···노동이 녹아내린다,이 땅에서 ‘비정규직’이라는 표현이 광범위하게 쓰인 지 20여년. 정부가 신규 일자...,2020-01-01
1,일자리 아닌 일거리···'일정치 않음'을 살다 [녹아내리는 노동],ㆍ①일자리보다 일거리\n\n노동의 미래를 모두 알 수는 없다. 다만 현실에서 일어나...,2020-01-01
2,이곳은…기계의 정확도 높이는 ‘인간부품’ 육성공장 아닐까 [녹아내리는 노동],ㆍ①일자리보다 일거리\n\nㆍAI 학습데이터 가공 기업 ‘크라우드웍스’ 교육장 참관...,2020-01-01
3,"지혜씨, 지훈씨를 부탁해! [인포그래픽]",새로 태어나는 아이들은 줄어들고 어르신들은 계속 나이를 먹어 갑니다. 2020년 새...,2020-01-01
4,"[20대 리포트] “20대, 함부로 판단말라… 내 길 내가 간다”",새천년의 시작과 함께 세상에 나온 2000년대생들이 어느덧 사회 변화를 이끌 20대...,2020-01-01
...,...,...,...
29036,"""2.5단계로 무급휴가 요구""… 코로나 재유행에 살아난 '갑질'",직장인 A씨는 수도권 사회적 거리두기 2.5단계를 시행하는 30일부터 다음달 6일...,2020-08-31
29037,모호한 조치ㆍ 깜깜이 환자 비율 40%육박... 거리두기 2.5단계 효과 있을까,"30일 0시부터 서울 전역에 강화된 사회적 거리두기(2.5단계)가 시행됐지만, 전문...",2020-08-31
29038,내일부터 서울 시내버스 밤 9시 이후 20% 감축… 주말 수준으로,수도권의 ‘사회적 거리두기’ 2.5단계 시행 기간에 맞춰 서울 시내버스의 야간 운행...,2020-08-31
29039,집단휴업 두고 전공의 vs 정부 '강대강 대치'... 의료공백 장기화 불가피,21일부터 10일째 무기한 집단휴진을 이어가고 있는 전공의(인턴ㆍ레지던트)들과 정부...,2020-08-31


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29041 entries, 0 to 29040
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   29041 non-null  object
 1   text    29041 non-null  object
 2   date    29041 non-null  object
dtypes: object(3)
memory usage: 680.8+ KB


In [39]:
# 한글,공백 제외하고 제거
df['title'] = df['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['text'] = df['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df.sample(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,title,text
15629,리쇼어링 유인책 늘려도 업계 수도권 입지 규제 등 실효성 적어,대통령 리쇼어링 활성화 산업계는 부정적 시각해외공장 유턴 법인세임대료 혜택 수도권엔...
21786,수도권 부지 곳에 청년신혼부부 공공주택,군 관사와 함께 개발 정부가 위례와 서울 남태령 동작구 수방사 군 부지를 청년신...
8924,미래한국당 비례수정안 수용불가 한선교 추가 조정 쉽지않아,민주는 비례정당 창당 싸고 정개련 등 시민단체와 충돌황교안 미래통합당 대표가 일 비...
9710,한밤 다급히 찾은 트럼프 의료장비 지원해달라,도널드 트럼프 미국 대통령이 일 밤 문재인 대통령과의 전화 통화에서 한국에 신종 코...
27344,전광훈 목사 코로나 확진 판정 방역 당국 자가격리 위반,지난 일 서울 광화문 집회에 참석한 사랑제일교회 전광훈 담임목사를 재수감해야 한다는...
17404,어린이 코로나합병증 의심 환자 명 올 초 필리핀 체류,서울신문방역당국 다음주 초 관련성 여부 등 발표 의료계 코로나와 연관성 불명확 신중...
8361,안전자산 채권마저 폭락,현물 당 가격 하락 일부 회사채 금리 넘나들어환율 또 원 넘게 올라 원 주식채...
9258,비례당 공천위원 명 보니 역시 조국,총선 조국 지지 교수 성명 낸 김호범작가 성명 참여한 소설가 정도상민주당 공천 실무...
4007,년 월 일 김용민의 그림마당,경향신문 무단전재 및 재배포 금지
27587,년 전 품격 강조했던 미셸 이번엔 이 악물고 트럼프 비판,년 월과 같은 관중의 열렬한 환호와 박수는 없었다 목소리 톤은 더 낮아졌고 어조는 ...


In [46]:
okt = Okt()

## title 명사 추출

In [78]:
title_nouns = []
for sentence in tqdm(df['title']):
    tokenized_sentence = okt.nouns(sentence) # 명사만 추출
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    title_nouns.append(stopwords_removed_sentence)

100%|██████████| 29041/29041 [06:32<00:00, 73.96it/s]


In [92]:
# 한글자 (년,명,월 등)은 제외
title_nouns_list = []
for li in title_nouns:
    for n in li:
        if len(n)>1:
            title_nouns_list.append(n)

In [95]:
# 명사 빈도
count = Counter(title_nouns_list)
title_nouns_cnt = count.most_common(20)
for i in title_nouns_cnt:
    print(i)

('코로나', 2912)
('정부', 1194)
('대통령', 1081)
('감염', 998)
('통합', 989)
('한국', 976)
('확산', 935)
('비상', 912)
('단독', 752)
('확진', 697)
('총선', 690)
('민주당', 652)
('국민', 610)
('수사', 558)
('진자', 538)
('경제', 531)
('트럼프', 517)
('방역', 514)
('우한', 491)
('환자', 486)
